In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import os
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.metrics.cluster import homogeneity_score
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

In [2]:
df_metrics = pd.DataFrame(columns=['ARI_Louvain','ARI_kmeans','ARI_HC',
                                   'AMI_Louvain','AMI_kmeans','AMI_HC',
                                   'Homogeneity_Louvain','Homogeneity_kmeans','Homogeneity_HC'])

In [3]:
workdir = './output_new/'
path_fm = os.path.join(workdir,'feature_matrices/')
path_clusters = os.path.join(workdir,'clusters/')
path_metrics = os.path.join(workdir,'metrics/')
os.system('mkdir -p '+path_clusters)
os.system('mkdir -p '+path_metrics)

0

In [4]:
metadata = pd.read_csv('../input/metadata.tsv',sep='\t',index_col=0)
num_clusters = len(np.unique(metadata['label']))
print(num_clusters)

10


In [5]:
files = [x for x in os.listdir(path_fm) if x.startswith('fm')]
len(files)

18

In [6]:
files

['fm_Control.rds',
 'fm_Control_new.rds',
 'fm_BROCKMAN.rds',
 'fm_BROCKMAN_new.rds',
 'fm_Cusanovich2018.rds',
 'fm_Cusanovich2018_new.rds',
 'fm_chromVAR_kmers2.rds',
 'fm_chromVAR_kmers2_new.rds',
 'fm_chromVAR_motifs2.rds',
 'fm_chromVAR_motifs2_new.rds',
 'fm_GeneScoring2.rds',
 'fm_GeneScoring2_new.rds',
 'fm_Cicero2.rds',
 'fm_Cicero2_new.rds',
 'fm_SnapATAC.rds',
 'fm_SnapATAC_new.rds',
 'fm_SCRAT2.rds',
 'fm_SCRAT2_new.rds']

In [7]:
def getNClusters(adata,n_cluster,range_min=0,range_max=3,max_steps=20):
    this_step = 0
    this_min = float(range_min)
    this_max = float(range_max)
    while this_step < max_steps:
        print('step ' + str(this_step))
        this_resolution = this_min + ((this_max-this_min)/2)
        sc.tl.louvain(adata,resolution=this_resolution)
        this_clusters = adata.obs['louvain'].nunique()
        
        print('got ' + str(this_clusters) + ' at resolution ' + str(this_resolution))
        
        if this_clusters > n_cluster:
            this_max = this_resolution
        elif this_clusters < n_cluster:
            this_min = this_resolution
        else:
            return(this_resolution, adata)
        this_step += 1
    
    print('Cannot find the number of clusters')
    print('Clustering solution from last iteration is used:' + str(this_clusters) + ' at resolution ' + str(this_resolution))

In [8]:
for file in files:
    file_split = file[:-4].split('_')
    method = file_split[1]
    if(len(file_split)>2):
        method = method + '_'+''.join(file_split[2:]).replace('2','_pca')
    print(method)

    pandas2ri.activate()
    readRDS = robjects.r['readRDS']
    df_rds = readRDS(os.path.join(path_fm,file))
    fm_mat = pandas2ri.ri2py(robjects.r['data.frame'](robjects.r['as.matrix'](df_rds)))
    fm_mat.fillna(0,inplace=True)
    fm_mat.columns = metadata.index
    
    adata = sc.AnnData(fm_mat.T)
    adata.var_names_make_unique()
    adata.obs = metadata.loc[adata.obs.index,]
    df_metrics.loc[method,] = ""
    #Louvain
    sc.pp.neighbors(adata, n_neighbors=15,use_rep='X')
#     sc.tl.louvain(adata)
    getNClusters(adata,n_cluster=num_clusters)
    #kmeans
    kmeans = KMeans(n_clusters=num_clusters, random_state=2019).fit(adata.X)
    adata.obs['kmeans'] = pd.Series(kmeans.labels_,index=adata.obs.index).astype('category')
    #hierachical clustering
    hc = AgglomerativeClustering(n_clusters=num_clusters).fit(adata.X)
    adata.obs['hc'] = pd.Series(hc.labels_,index=adata.obs.index).astype('category')
    #clustering metrics
    
    #adjusted rank index
    ari_louvain = adjusted_rand_score(adata.obs['label'], adata.obs['louvain'])
    ari_kmeans = adjusted_rand_score(adata.obs['label'], adata.obs['kmeans'])
    ari_hc = adjusted_rand_score(adata.obs['label'], adata.obs['hc'])
    #adjusted mutual information
    ami_louvain = adjusted_mutual_info_score(adata.obs['label'], adata.obs['louvain'],average_method='arithmetic')
    ami_kmeans = adjusted_mutual_info_score(adata.obs['label'], adata.obs['kmeans'],average_method='arithmetic')   
    ami_hc = adjusted_mutual_info_score(adata.obs['label'], adata.obs['hc'],average_method='arithmetic')
    #homogeneity
    homo_louvain = homogeneity_score(adata.obs['label'], adata.obs['louvain'])
    homo_kmeans = homogeneity_score(adata.obs['label'], adata.obs['kmeans'])
    homo_hc = homogeneity_score(adata.obs['label'], adata.obs['hc'])

    df_metrics.loc[method,['ARI_Louvain','ARI_kmeans','ARI_HC']] = [ari_louvain,ari_kmeans,ari_hc]
    df_metrics.loc[method,['AMI_Louvain','AMI_kmeans','AMI_HC']] = [ami_louvain,ami_kmeans,ami_hc]
    df_metrics.loc[method,['Homogeneity_Louvain','Homogeneity_kmeans','Homogeneity_HC']] = [homo_louvain,homo_kmeans,homo_hc] 
    adata.obs[['louvain','kmeans','hc']].to_csv(os.path.join(path_clusters ,method + '_clusters.tsv'),sep='\t')

Control


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 19 at resolution 1.5
step 1
got 12 at resolution 0.75
step 2
got 5 at resolution 0.375
step 3
got 10 at resolution 0.5625
Control_new


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 14 at resolution 1.5
step 1
got 9 at resolution 0.75
step 2
got 12 at resolution 1.125
step 3
got 10 at resolution 0.9375
BROCKMAN


...

step 0
got 22 at resolution 1.5
step 1
got 17 at resolution 0.75
step 2
got 15 at resolution 0.375
step 3
got 9 at resolution 0.1875
step 4
got 13 at resolution 0.28125
step 5
got 12 at resolution 0.234375
step 6
got 11 at resolution 0.2109375
step 7
got 11 at resolution 0.19921875
step 8
got 10 at resolution 0.193359375
SCRAT2


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 18 at resolution 1.5
step 1
got 9 at resolution 0.75
step 2
got 15 at resolution 1.125
step 3
got 13 at resolution 0.9375
step 4
got 13 at resolution 0.84375
step 5
got 9 at resolution 0.796875
step 6
got 12 at resolution 0.8203125
step 7
got 10 at resolution 0.80859375
SCRAT2_new


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 17 at resolution 1.5
step 1
got 10 at resolution 0.75


In [9]:
df_metrics.to_csv(path_metrics+'clustering_scores.csv')

In [10]:
df_metrics

,ARI_Louvain,ARI_kmeans,ARI_HC,AMI_Louvain,AMI_kmeans,AMI_HC,Homogeneity_Louvain,Homogeneity_kmeans,Homogeneity_HC
Control,0.182422,0.0137586,0.0344167,0.372463,0.024063,0.0449997,0.384145,0.0242291,0.044414
Control_new,0.341632,0.0644169,0.0509295,0.515555,0.223223,0.199813,0.531536,0.17637,0.118999
BROCKMAN,0.115213,0.0329884,0.0353507,0.261593,0.103987,0.119018,0.274468,0.115736,0.131517
BROCKMAN_new,0.212068,0.132393,0.120123,0.392578,0.267385,0.253048,0.408518,0.269279,0.255607
Cusanovich2018,0.533643,-0.00194164,-0.00194164,0.663841,-0.00278841,-0.00278841,0.680403,0.00302188,0.00302188
Cusanovich2018_new,0.528593,-0.00194164,-0.00194164,0.6594,-0.00278841,-0.00278841,0.676079,0.00302188,0.00302188
chromVAR_kmers_pca,0.443234,0.262985,0.270338,0.56954,0.420358,0.434644,0.590846,0.377882,0.386469
chromVAR_kmers_pcanew,0.306888,0.170346,0.049337,0.461563,0.287678,0.0853234,0.479484,0.241079,0.0623269
chromVAR_motifs_pca,0.308443,0.256185,0.294866,0.516459,0.410229,0.43911,0.534353,0.407546,0.412825
chromVAR_motifs_pcanew,0.233923,0.161979,0.120613,0.414829,0.334581,0.279056,0.432391,0.328071,0.236596
